In [1]:
###-----------------
### Import Libraries
###-----------------

import os

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay


import tensorflow as tf

    
%matplotlib inline

In [2]:
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
###---------------------------------------
### Global  parameters and Hyperparamaters
###---------------------------------------

inpDir = '/content/drive/MyDrive/datasets' # location where input data is stored
outDir = '/content/drive/MyDrive/outputDir' # location to store outputs
subDir = '/kaggle/input/adience-dataset-preprocessed/datasets/age' # location of the images
modelDir = '/kaggle/input/adience-dataset-preprocessed/datasets/gender' # location to save model files
valDir = '/content/drive/MyDrive/datasets/gender' # location related to this dataset
#altName = 'dropout_bn_no_bias_leaky_flowers' # Model file name for this alternative

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
tf.random.set_seed(RANDOM_STATE)


EPOCHS = 10   # number of cycles to run

ALPHA = 0.01

BATCH_SIZE = 16

TEST_SIZE = 0.2

IMG_HEIGHT = 224

IMG_WIDTH = 224

FLIP_MODE = "horizontal_and_vertical"

# for rotation transformation 
ROTATION_FACTOR = (-0.1, 0.1) 

FILL_MODE = 'nearest'

ES_PATIENCE = 20 # if performance does not improve stop

LR_PATIENCE = 10 # if performace is not improving reduce alpha

LR_FACTOR = 0.5 # rate of reduction of alpha# Train the model
num_epochs = 10

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,6),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.brg

plt.rcParams.update(params) # update rcParams

In [4]:
data_dir = os.path.join(inpDir, subDir)

In [5]:
data_dir2 = os.path.join(inpDir, modelDir)

In [6]:
data_dir

'/kaggle/input/adience-dataset-preprocessed/datasets/age'

In [7]:
data_dir2

'/kaggle/input/adience-dataset-preprocessed/datasets/gender'

In [8]:
# list content of data_dir
os.listdir(data_dir)

['25-36', '60-100', '8-13', '0-3', '15-23', '38-48', '48-58', '4-6']

In [9]:
os.listdir(data_dir2)

['f', 'm']

In [10]:
Input_Shape = (IMG_HEIGHT, IMG_WIDTH, 3)

In [11]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=TEST_SIZE,
    subset='training',
    seed= RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=TEST_SIZE,
    subset='validation',
    seed= RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

Found 11030 files belonging to 8 classes.
Using 8824 files for training.
Found 11030 files belonging to 8 classes.
Using 2206 files for validation.


In [12]:
train_ds2 = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir2,
    validation_split=TEST_SIZE,
    subset='training',
    seed= RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

test_ds2 = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir2,
    validation_split=TEST_SIZE,
    subset='validation',
    seed= RANDOM_STATE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

Found 9848 files belonging to 2 classes.
Using 7879 files for training.
Found 9848 files belonging to 2 classes.
Using 1969 files for validation.


In [13]:
class_names = train_ds.class_names
print('total ', len(class_names), class_names)

total  8 ['0-3', '15-23', '25-36', '38-48', '4-6', '48-58', '60-100', '8-13']


In [14]:
class_names2 = train_ds2.class_names
print('total ', len(class_names2), class_names2)

total  2 ['f', 'm']


In [15]:
## Optimize for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
## Optimize for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds2.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds2.cache().prefetch(buffer_size=AUTOTUNE)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
age_model = Sequential()
age_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
age_model.add(MaxPooling2D(2, 2))
age_model.add(Conv2D(64, (3, 3), activation='relu'))
age_model.add(MaxPooling2D(2, 2))
age_model.add(Conv2D(128, (3, 3), activation='relu'))
age_model.add(MaxPooling2D(2, 2))
age_model.add(Flatten())
age_model.add(Dense(128, activation='relu'))
age_model.add(Dropout(0.5))
age_model.add(Dense(1, activation='linear', name='age'))

age_model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae', 'accuracy'])
age_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 26, 26, 128)     

In [23]:
history_age = age_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)

Epoch 1/10
493/493 [==============================] - 17s 33ms/step - loss: 6156.3008 - mae: 4.8640 - accuracy: 0.5403 - val_loss: 0.2549 - val_mae: 0.4696 - val_accuracy: 0.5724
Epoch 2/10
493/493 [==============================] - 16s 33ms/step - loss: 0.2974 - mae: 0.4579 - accuracy: 0.5718 - val_loss: 0.2557 - val_mae: 0.4579 - val_accuracy: 0.5714
Epoch 3/10
493/493 [==============================] - 16s 33ms/step - loss: 0.2770 - mae: 0.4416 - accuracy: 0.5939 - val_loss: 0.2478 - val_mae: 0.4499 - val_accuracy: 0.5891
Epoch 4/10
493/493 [==============================] - 16s 33ms/step - loss: 0.2710 - mae: 0.4319 - accuracy: 0.6092 - val_loss: 0.2372 - val_mae: 0.4420 - val_accuracy: 0.6140
Epoch 5/10
493/493 [==============================] - 16s 33ms/step - loss: 0.2576 - mae: 0.4206 - accuracy: 0.6261 - val_loss: 0.2453 - val_mae: 0.4326 - val_accuracy: 0.6191
Epoch 6/10
493/493 [==============================] - 16s 33ms/step - loss: 0.2521 - mae: 0.4137 - accuracy: 0.6317 -

In [24]:
age_loss, age_mae, age_accuracy = age_model.evaluate(test_ds)
print(f"Age Prediction Model - Mean Absolute Error: {age_mae}, Accuracy: {age_accuracy}")

124/124 [==============================] - 2s 14ms/step - loss: 0.2465 - mae: 0.4437 - accuracy: 0.6079
Age Prediction Model - Mean Absolute Error: 0.4436931908130646, Accuracy: 0.6079227924346924


In [25]:
gender_model = Sequential()
gender_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
gender_model.add(MaxPooling2D(2, 2))
gender_model.add(Conv2D(64, (3, 3), activation='relu'))
gender_model.add(MaxPooling2D(2, 2))
gender_model.add(Conv2D(128, (3, 3), activation='relu'))
gender_model.add(MaxPooling2D(2, 2))
gender_model.add(Flatten())
gender_model.add(Dense(128, activation='relu'))
gender_model.add(Dropout(0.5))
gender_model.add(Dense(1, activation='sigmoid', name='gender'))

gender_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
history_gender = gender_model.fit(
    train_ds2,
    validation_data=test_ds2,
    epochs=EPOCHS
)

Epoch 1/10
493/493 [==============================] - 25s 48ms/step - loss: 1.9837 - accuracy: 0.5328 - val_loss: 0.6922 - val_accuracy: 0.5231
Epoch 2/10
493/493 [==============================] - 22s 44ms/step - loss: 0.6889 - accuracy: 0.5475 - val_loss: 0.7053 - val_accuracy: 0.5033
Epoch 3/10
493/493 [==============================] - 22s 45ms/step - loss: 0.6771 - accuracy: 0.5667 - val_loss: 0.7714 - val_accuracy: 0.5119
Epoch 4/10
493/493 [==============================] - 22s 45ms/step - loss: 0.6558 - accuracy: 0.6011 - val_loss: 0.7318 - val_accuracy: 0.5058
Epoch 5/10
493/493 [==============================] - 22s 44ms/step - loss: 0.6363 - accuracy: 0.6252 - val_loss: 0.7526 - val_accuracy: 0.5282
Epoch 6/10
493/493 [==============================] - 22s 44ms/step - loss: 0.6091 - accuracy: 0.6535 - val_loss: 0.8127 - val_accuracy: 0.5160
Epoch 7/10
493/493 [==============================] - 22s 44ms/step - loss: 0.6124 - accuracy: 0.6456 - val_loss: 0.7713 - val_accuracy:

In [27]:
gender_loss, gender_accuracy = gender_model.evaluate(test_ds2)
print(f"Gender Prediction Model - Accuracy: {gender_accuracy}")

124/124 [==============================] - 4s 32ms/step - loss: 0.9573 - accuracy: 0.5312
Gender Prediction Model - Accuracy: 0.5312341451644897
